# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 13 2023 1:56PM,264679,18,SEQ-9569226,Seqirus,Released
1,Jun 13 2023 1:48PM,264678,15,30017241,"Alliance Pharma, Inc.",Released
2,Jun 13 2023 1:45PM,264677,16,SSF_C1W24.1,Sartorius Stedim Filters Inc.,Released
3,Jun 13 2023 1:36PM,264675,15,PNC459015B,"Person & Covey, Inc.",Released
4,Jun 13 2023 1:36PM,264675,15,PNC459020B,"Person & Covey, Inc.",Released
5,Jun 13 2023 1:36PM,264675,15,PNC459023B,"Person & Covey, Inc.",Released
6,Jun 13 2023 1:36PM,264675,15,PNC459196A,"Person & Covey, Inc.",Released
7,Jun 13 2023 1:36PM,264675,15,PNC459199A,"Person & Covey, Inc.",Released
8,Jun 13 2023 1:36PM,264675,15,PNC459144A,"Person & Covey, Inc.",Released
9,Jun 13 2023 1:36PM,264675,15,PNC459146A,"Person & Covey, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
16,264675,Released,29
17,264676,Released,1
18,264677,Released,1
19,264678,Released,1
20,264679,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
264675,NaN,NaN,29.0
264676,NaN,NaN,1.0
264677,NaN,NaN,1.0
264678,NaN,NaN,1.0
264679,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
264620,7.0,8.0,1.0
264654,5.0,16.0,5.0
264661,0.0,19.0,9.0
264663,0.0,2.0,1.0
264664,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
264620,7,8,1
264654,5,16,5
264661,0,19,9
264663,0,2,1
264664,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,264620,7,8,1
1,264654,5,16,5
2,264661,0,19,9
3,264663,0,2,1
4,264664,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,264620,7,8,1
1,264654,5,16,5
2,264661,,19,9
3,264663,,2,1
4,264664,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 13 2023 1:56PM,264679,18,Seqirus
1,Jun 13 2023 1:48PM,264678,15,"Alliance Pharma, Inc."
2,Jun 13 2023 1:45PM,264677,16,Sartorius Stedim Filters Inc.
3,Jun 13 2023 1:36PM,264675,15,"Person & Covey, Inc."
32,Jun 13 2023 1:35PM,264676,10,Hush Hush
33,Jun 13 2023 1:34PM,264674,10,Hush Hush
39,Jun 13 2023 1:26PM,264673,10,"Methapharm, Inc."
42,Jun 13 2023 1:22PM,264672,10,Methapharm-G
43,Jun 13 2023 12:45PM,264669,10,"Citieffe, Inc."
45,Jun 13 2023 12:40PM,264666,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 13 2023 1:56PM,264679,18,Seqirus,,,1
1,Jun 13 2023 1:48PM,264678,15,"Alliance Pharma, Inc.",,,1
2,Jun 13 2023 1:45PM,264677,16,Sartorius Stedim Filters Inc.,,,1
3,Jun 13 2023 1:36PM,264675,15,"Person & Covey, Inc.",,,29
4,Jun 13 2023 1:35PM,264676,10,Hush Hush,,,1
5,Jun 13 2023 1:34PM,264674,10,Hush Hush,,,6
6,Jun 13 2023 1:26PM,264673,10,"Methapharm, Inc.",,,3
7,Jun 13 2023 1:22PM,264672,10,Methapharm-G,,,1
8,Jun 13 2023 12:45PM,264669,10,"Citieffe, Inc.",,,2
9,Jun 13 2023 12:40PM,264666,19,"AdvaGen Pharma, Ltd",,2,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2023 1:56PM,264679,18,Seqirus,1,,
1,Jun 13 2023 1:48PM,264678,15,"Alliance Pharma, Inc.",1,,
2,Jun 13 2023 1:45PM,264677,16,Sartorius Stedim Filters Inc.,1,,
3,Jun 13 2023 1:36PM,264675,15,"Person & Covey, Inc.",29,,
4,Jun 13 2023 1:35PM,264676,10,Hush Hush,1,,
5,Jun 13 2023 1:34PM,264674,10,Hush Hush,6,,
6,Jun 13 2023 1:26PM,264673,10,"Methapharm, Inc.",3,,
7,Jun 13 2023 1:22PM,264672,10,Methapharm-G,1,,
8,Jun 13 2023 12:45PM,264669,10,"Citieffe, Inc.",2,,
9,Jun 13 2023 12:40PM,264666,19,"AdvaGen Pharma, Ltd",,2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2023 1:56PM,264679,18,Seqirus,1,,
1,Jun 13 2023 1:48PM,264678,15,"Alliance Pharma, Inc.",1,,
2,Jun 13 2023 1:45PM,264677,16,Sartorius Stedim Filters Inc.,1,,
3,Jun 13 2023 1:36PM,264675,15,"Person & Covey, Inc.",29,,
4,Jun 13 2023 1:35PM,264676,10,Hush Hush,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2023 1:56PM,264679,18,Seqirus,1.0,NaN,NaN
1,Jun 13 2023 1:48PM,264678,15,"Alliance Pharma, Inc.",1.0,NaN,NaN
2,Jun 13 2023 1:45PM,264677,16,Sartorius Stedim Filters Inc.,1.0,NaN,NaN
3,Jun 13 2023 1:36PM,264675,15,"Person & Covey, Inc.",29.0,NaN,NaN
4,Jun 13 2023 1:35PM,264676,10,Hush Hush,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2023 1:56PM,264679,18,Seqirus,1.0,0.0,0.0
1,Jun 13 2023 1:48PM,264678,15,"Alliance Pharma, Inc.",1.0,0.0,0.0
2,Jun 13 2023 1:45PM,264677,16,Sartorius Stedim Filters Inc.,1.0,0.0,0.0
3,Jun 13 2023 1:36PM,264675,15,"Person & Covey, Inc.",29.0,0.0,0.0
4,Jun 13 2023 1:35PM,264676,10,Hush Hush,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1323364,13.0,0.0,0.0
15,1058677,40.0,21.0,0.0
16,264677,1.0,0.0,0.0
18,264679,1.0,0.0,0.0
19,793940,6.0,26.0,12.0
20,264664,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1323364,13.0,0.0,0.0
1,15,1058677,40.0,21.0,0.0
2,16,264677,1.0,0.0,0.0
3,18,264679,1.0,0.0,0.0
4,19,793940,6.0,26.0,12.0
5,20,264664,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,13.0,0.0,0.0
1,15,40.0,21.0,0.0
2,16,1.0,0.0,0.0
3,18,1.0,0.0,0.0
4,19,6.0,26.0,12.0
5,20,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,13.0
1,15,Released,40.0
2,16,Released,1.0
3,18,Released,1.0
4,19,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20
Status,,,,,,
Completed,0.0,0.0,0.0,0.0,12.0,0.0
Executing,0.0,21.0,0.0,0.0,26.0,0.0
Released,13.0,40.0,1.0,1.0,6.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20
0,Completed,0.0,0.0,0.0,0.0,12.0,0.0
1,Executing,0.0,21.0,0.0,0.0,26.0,0.0
2,Released,13.0,40.0,1.0,1.0,6.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20
0,Completed,0.0,0.0,0.0,0.0,12.0,0.0
1,Executing,0.0,21.0,0.0,0.0,26.0,0.0
2,Released,13.0,40.0,1.0,1.0,6.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()